# Statistical modelling of coffee desk data

## Import and read data from csv file

In [30]:
!pip install statsmodels

  Using cached patsy-0.5.1-py2.py3-none-any.whl (231 kB)


In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as smf

In [153]:
coffee_df = pd.read_csv('data\coffee_desk_dataset_ead.csv')
coffee_df.drop(columns='idx', axis=1, inplace=True) #dropping index not to be treated as vector dimension
coffee_df

,process,brewing method,roast,grind,origin,price_per_kg,arabica (%),Pure arabica,roast_brew,Washed,Natural,Fermented/macerated (traditional),Fermented/macerated (closed tank),process_general,region of origin
0,Monsooning,drip (alternative brewing methods),light,beans,Laos,52.22,100,True,light_drip (alternative brewing methods),False,False,False,False,Monsooning,Asia
1,Natural,drip (alternative brewing methods),medium,beans,Brazylia,31.92,100,True,medium_drip (alternative brewing methods),False,True,False,False,Natural,Latam
2,Natural,drip (alternative brewing methods),light,beans,Etiopia,39.20,100,True,light_drip (alternative brewing methods),False,True,False,False,Natural,Africa
3,Washed,drip (alternative brewing methods),light,beans,Etiopia,39.20,100,True,light_drip (alternative brewing methods),True,False,False,False,Washed,Africa
4,Natural,drip (alternative brewing methods),dark,beans,Indonezja,35.20,100,True,dark_drip (alternative brewing methods),False,True,False,False,Natural,Asia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,Rum Aged,drip (alternative brewing methods),light,beans,Gwatemala,73.33,100,True,light_drip (alternative brewing methods),False,False,False,False,Fermented,Latam
858,Natural,espresso,light,beans,Panama,50.00,30,False,light_espresso,False,True,False,False,Natural,Latam
859,Pulped natural,drip (alternative brewing methods),light,beans,Nikaragua,36.00,100,True,light_drip (alternative brewing methods),True,True,False,False,Hybrid,Latam
860,Washed,drip (alternative brewing methods),light,beans,Gwatemala,25.00,100,True,light_drip (alternative brewing methods),True,False,False,False,Washed,Latam


In [181]:
coffee_df.rename(columns={'brewing method':'brewing_method', 'arabica (%)':'percentage_of_arabica', 'Pure arabica':'pure_arabica', 'region of origin':'region_of_origin', 'Fermented/macerated (traditional)':'Fermented_traditional', 'Fermented/macerated (closed tank)':'Fermented_closedtank'}, inplace=True)

In [328]:
coffee_df = coffee_df[coffee_df.price_per_kg != 135.2]
coffee_df = coffee_df[coffee_df.price_per_kg != 71.88]
coffee_df = coffee_df[coffee_df.price_per_kg < 100]

In [329]:
region_fig = px.violin(coffee_df, x='region_of_origin', y='price_per_kg', box=True, points="all", title="Coffee region of origin vs coffee price")
region_fig.update_layout(xaxis_type="category", xaxis={'categoryorder':'mean ascending'})

## Train-val-test split

In [330]:
X_df = coffee_df.drop('price_per_kg', axis=1) #defining predictors
y_df = coffee_df['price_per_kg'] #defining target variable

In [331]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.1, random_state=True) #using random state to ensure I always have random division with the same random numbers

In [332]:
train_df = pd.concat([y_train, X_train], axis=1)
train_df.head()

,price_per_kg,process,brewing_method,roast,grind,origin,percentage_of_arabica,pure_arabica,roast_brew,Washed,Natural,Fermented_traditional,Fermented_closedtank,process_general,region_of_origin
837,39.2,Natural,espresso,medium,beans,Etiopia,100,True,medium_espresso,False,True,False,False,Natural,Africa
697,24.0,Natural,espresso,medium,beans,Brazylia,100,True,medium_espresso,False,True,False,False,Natural,Latam
96,44.0,Washed,drip (alternative brewing methods),light,beans,Rwanda,100,True,light_drip (alternative brewing methods),True,False,False,False,Washed,Africa
677,20.8,Washed,espresso,medium,beans,"Peru, Gwatemala",100,True,medium_espresso,True,False,False,False,Washed,Latam
823,23.8,Semi-Washed Kombucha Experiment,espresso,dark,beans,"Brazylia, Indie",100,True,dark_espresso,True,False,False,True,Fermented,Mixed


## Initial stats modelling

## Forward stepwise selection
1. Begins with a model that contains no variables (called the Null Model)
2. Then starts adding the most significant variables one after the other
3. Until a pre-specified stopping rule is reached or until all the variables under consideration are included in the model

Steps:
* determine the most significant variables:
    - has the smallest p-value
    - provides the highest increase in R^2
    - provides the highest drop in modell RSS (Residuals Sum of Squares)
* choose a stopping rule

In [333]:
model_0 = smf.ols(formula="price_per_kg ~ 1", data=train_df).fit() # the initial model with no variables
print(model_0.summary())

                            OLS Regression Results                            
Dep. Variable:           price_per_kg   R-squared:                      -0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Mon, 31 May 2021   Prob (F-statistic):                nan
Time:                        21:53:41   Log-Likelihood:                -3168.0
No. Observations:                 753   AIC:                             6338.
Df Residuals:                     752   BIC:                             6343.
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     34.5084      0.593     58.224      0.0

In [334]:
model_0.ssr # sum in squares for the original model

198914.01561938913

In [335]:
# process - not such a good variable, high granularity and high pvalues
process_model = smf.ols(formula="price_per_kg ~ C(process)", data=train_df).fit()
print(process_model.summary())
process_model.ssr

                            OLS Regression Results                            
Dep. Variable:           price_per_kg   R-squared:                       0.205
Model:                            OLS   Adj. R-squared:                  0.179
Method:                 Least Squares   F-statistic:                     7.829
Date:                Mon, 31 May 2021   Prob (F-statistic):           6.91e-24
Time:                        21:53:48   Log-Likelihood:                -3081.6
No. Observations:                 753   AIC:                             6213.
Df Residuals:                     728   BIC:                             6329.
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                                    coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------

158107.86030682793

In [336]:
# roast_brew - higher p-value -> not such a good variable
roast_brew_model = smf.ols(formula="price_per_kg ~ C(roast_brew)", data=train_df).fit()
print(roast_brew_model.summary())
roast_brew_model.ssr

                            OLS Regression Results                            
Dep. Variable:           price_per_kg   R-squared:                       0.432
Model:                            OLS   Adj. R-squared:                  0.426
Method:                 Least Squares   F-statistic:                     70.70
Date:                Mon, 31 May 2021   Prob (F-statistic):           3.16e-86
Time:                        21:53:52   Log-Likelihood:                -2955.1
No. Observations:                 753   AIC:                             5928.
Df Residuals:                     744   BIC:                             5970.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                                                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------

113005.03734407586

In [248]:
process_general_model.ssr

348018.5580086071

In [345]:
# Fermented_closedtank - low p-value -> not a good variable AND low r-squared
Fermented_closedtank_model = smf.ols(formula="price_per_kg ~ C(Fermented_closedtank)", data=train_df).fit()
print(Fermented_closedtank_model.summary())
Fermented_closedtank_model.ssr

                            OLS Regression Results                            
Dep. Variable:           price_per_kg   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.3679
Date:                Mon, 31 May 2021   Prob (F-statistic):              0.544
Time:                        21:55:01   Log-Likelihood:                -3167.9
No. Observations:                 753   AIC:                             6340.
Df Residuals:                     751   BIC:                             6349.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept 

198816.61736076756

In [347]:
# process_general - low p-value -> not such a good good variable
process_general_model = smf.ols(formula="price_per_kg ~ C(process_general)", data=train_df).fit()
print(process_general_model.summary())
process_general_model.ssr

                            OLS Regression Results                            
Dep. Variable:           price_per_kg   R-squared:                       0.073
Model:                            OLS   Adj. R-squared:                  0.063
Method:                 Least Squares   F-statistic:                     7.271
Date:                Mon, 31 May 2021   Prob (F-statistic):           2.54e-09
Time:                        21:55:50   Log-Likelihood:                -3139.7
No. Observations:                 753   AIC:                             6297.
Df Residuals:                     744   BIC:                             6339.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Inte

184489.34473003048

In [337]:
# percentage_of_arabica -> very high p-value, not a good variable!
percentage_of_arabica_model = smf.ols(formula="price_per_kg ~ percentage_of_arabica", data=train_df).fit()
print(percentage_of_arabica_model.summary())
percentage_of_arabica_model.ssr

                            OLS Regression Results                            
Dep. Variable:           price_per_kg   R-squared:                       0.102
Model:                            OLS   Adj. R-squared:                  0.101
Method:                 Least Squares   F-statistic:                     85.69
Date:                Mon, 31 May 2021   Prob (F-statistic):           2.16e-19
Time:                        21:54:03   Log-Likelihood:                -3127.4
No. Observations:                 753   AIC:                             6259.
Df Residuals:                     751   BIC:                             6268.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 3.16

178542.98170310314

In [348]:
# origin   - low p-value -> potentially good variable, BUT it has high granularuty
origin_model = smf.ols(formula="price_per_kg ~ C(origin)", data=train_df).fit()
print(origin_model.summary())
origin_model.ssr

                            OLS Regression Results                            
Dep. Variable:           price_per_kg   R-squared:                       0.363
Model:                            OLS   Adj. R-squared:                  0.314
Method:                 Least Squares   F-statistic:                     7.507
Date:                Mon, 31 May 2021   Prob (F-statistic):           6.51e-41
Time:                        21:56:16   Log-Likelihood:                -2998.4
No. Observations:                 753   AIC:                             6105.
Df Residuals:                     699   BIC:                             6355.
Df Model:                          53                                         
Covariance Type:            nonrobust                                         
                                                  coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

126764.0238601965

In [349]:
# Washed - low p-value -> potentially good variable BUT low r-squared
Washed_model = smf.ols(formula="price_per_kg ~ C(Washed)", data=train_df).fit()
print(Washed_model.summary())
Washed_model.ssr

                            OLS Regression Results                            
Dep. Variable:           price_per_kg   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.5793
Date:                Mon, 31 May 2021   Prob (F-statistic):              0.447
Time:                        21:56:21   Log-Likelihood:                -3167.7
No. Observations:                 753   AIC:                             6339.
Df Residuals:                     751   BIC:                             6349.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            33.9749      0.91

198760.6932439933

In [351]:
# grind - low p-value -> potentially good variable BUT low r-squared
grind_model = smf.ols(formula="price_per_kg ~ C(grind)", data=train_df).fit()
print(grind_model.summary())
grind_model.ssr

                            OLS Regression Results                            
Dep. Variable:           price_per_kg   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     7.937
Date:                Mon, 31 May 2021   Prob (F-statistic):            0.00497
Time:                        21:56:30   Log-Likelihood:                -3164.1
No. Observations:                 753   AIC:                             6332.
Df Residuals:                     751   BIC:                             6341.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             35.0019      0

196833.84601352934

In [352]:
# pure_arabica - low p-value -> potentially good variable  BUT low r-squared
pure_arabica_model = smf.ols(formula="price_per_kg ~ C(pure_arabica)", data=train_df).fit()
print(pure_arabica_model.summary())
pure_arabica_model.ssr

                            OLS Regression Results                            
Dep. Variable:           price_per_kg   R-squared:                       0.143
Model:                            OLS   Adj. R-squared:                  0.142
Method:                 Least Squares   F-statistic:                     125.6
Date:                Mon, 31 May 2021   Prob (F-statistic):           4.62e-27
Time:                        21:56:36   Log-Likelihood:                -3109.8
No. Observations:                 753   AIC:                             6224.
Df Residuals:                     751   BIC:                             6233.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

170414.55725527884

In [353]:
# Natural  - low p-value -> potentially good variable BUT low r-squared
Natural_model = smf.ols(formula="price_per_kg ~ C(Natural)", data=train_df).fit()
print(Natural_model.summary())
Natural_model.ssr

                            OLS Regression Results                            
Dep. Variable:           price_per_kg   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     7.085
Date:                Mon, 31 May 2021   Prob (F-statistic):            0.00794
Time:                        21:56:40   Log-Likelihood:                -3164.5
No. Observations:                 753   AIC:                             6333.
Df Residuals:                     751   BIC:                             6342.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             35.9113      0

197054.94088519498

In [354]:
# Fermented_traditional - low p-value -> potentially good variable BUT low r-squared
Fermented_traditional_model = smf.ols(formula="price_per_kg ~ C(Fermented_traditional)", data=train_df).fit()
print(Fermented_traditional_model.summary())
Fermented_traditional_model.ssr

                            OLS Regression Results                            
Dep. Variable:           price_per_kg   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     20.50
Date:                Mon, 31 May 2021   Prob (F-statistic):           6.93e-06
Time:                        21:56:45   Log-Likelihood:                -3157.9
No. Observations:                 753   AIC:                             6320.
Df Residuals:                     751   BIC:                             6329.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

193628.48124708954

In [350]:
# brewing_method - very low p-value but not on all, to be tested now after removing outliers
brewing_method_model = smf.ols(formula="price_per_kg ~ C(brewing_method)", data=train_df).fit()
print(brewing_method_model.summary())
brewing_method_model.ssr

                            OLS Regression Results                            
Dep. Variable:           price_per_kg   R-squared:                       0.409
Model:                            OLS   Adj. R-squared:                  0.408
Method:                 Least Squares   F-statistic:                     260.0
Date:                Mon, 31 May 2021   Prob (F-statistic):           1.68e-86
Time:                        21:56:26   Log-Likelihood:                -2969.7
No. Observations:                 753   AIC:                             5945.
Df Residuals:                     750   BIC:                             5959.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
In

117472.06330342843

In [355]:
# region_of_origin - low p-value -> potentially good variable
region_of_origin_model = smf.ols(formula="price_per_kg ~ C(region_of_origin)", data=train_df).fit()
print(region_of_origin_model.summary())
region_of_origin_model.ssr

                            OLS Regression Results                            
Dep. Variable:           price_per_kg   R-squared:                       0.175
Model:                            OLS   Adj. R-squared:                  0.170
Method:                 Least Squares   F-statistic:                     39.59
Date:                Mon, 31 May 2021   Prob (F-statistic):           4.25e-30
Time:                        21:56:49   Log-Likelihood:                -3095.7
No. Observations:                 753   AIC:                             6201.
Df Residuals:                     748   BIC:                             6225.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept     

164157.0998470273

In [356]:
# roast - very low p-value -> potentially good variable AND has good r-squared AND low ssr
roast_model = smf.ols(formula="price_per_kg ~ C(roast)", data=train_df).fit()
print(roast_model.summary())
roast_model.ssr

                            OLS Regression Results                            
Dep. Variable:           price_per_kg   R-squared:                       0.369
Model:                            OLS   Adj. R-squared:                  0.368
Method:                 Least Squares   F-statistic:                     219.7
Date:                Mon, 31 May 2021   Prob (F-statistic):           7.89e-76
Time:                        21:56:58   Log-Likelihood:                -2994.4
No. Observations:                 753   AIC:                             5995.
Df Residuals:                     750   BIC:                             6009.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             21.5285      0

125426.95277673956

In [363]:
# Adding next best variable to the model
model_2 = smf.ols(formula="price_per_kg ~  C(brewing_method) + C(roast)", data=train_df).fit()
print(model_2.summary())
model_2.ssr

                            OLS Regression Results                            
Dep. Variable:           price_per_kg   R-squared:                       0.426
Model:                            OLS   Adj. R-squared:                  0.422
Method:                 Least Squares   F-statistic:                     138.5
Date:                Mon, 31 May 2021   Prob (F-statistic):           1.49e-88
Time:                        21:59:53   Log-Likelihood:                -2959.3
No. Observations:                 753   AIC:                             5929.
Df Residuals:                     748   BIC:                             5952.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
In

114271.32317079302

**Observations:** We see that adding region of origin to roast decreases srr of the model

In [362]:
# Adding next best variable to the model
model_3 = smf.ols(formula="price_per_kg ~ C(brewing_method) + C(roast) + C(pure_arabica)", data=train_df).fit()
print(model_3.summary())
model_3.ssr # again the ssr decreased

                            OLS Regression Results                            
Dep. Variable:           price_per_kg   R-squared:                       0.440
Model:                            OLS   Adj. R-squared:                  0.436
Method:                 Least Squares   F-statistic:                     117.2
Date:                Mon, 31 May 2021   Prob (F-statistic):           1.74e-91
Time:                        21:59:14   Log-Likelihood:                -2949.9
No. Observations:                 753   AIC:                             5912.
Df Residuals:                     747   BIC:                             5940.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
In

111453.17215187248

In [364]:
# Adding next best variable to the model
model_4 = smf.ols(formula="price_per_kg ~ C(brewing_method) + C(roast) + C(pure_arabica) + C(grind)", data=train_df).fit()
print(model_4.summary())
model_4.ssr # again the ssr decreased

                            OLS Regression Results                            
Dep. Variable:           price_per_kg   R-squared:                       0.442
Model:                            OLS   Adj. R-squared:                  0.437
Method:                 Least Squares   F-statistic:                     98.40
Date:                Mon, 31 May 2021   Prob (F-statistic):           5.01e-91
Time:                        22:00:36   Log-Likelihood:                -2948.5
No. Observations:                 753   AIC:                             5911.
Df Residuals:                     746   BIC:                             5943.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
In

111039.32822297813

In [367]:
# Adding next best variable to the model
model_5 = smf.ols(formula="price_per_kg ~ C(brewing_method) + C(roast) + C(pure_arabica) + C(grind) + C(region_of_origin)", data=train_df).fit()
print(model_5.summary())
model_5.ssr # again the ssr decreased 

                            OLS Regression Results                            
Dep. Variable:           price_per_kg   R-squared:                       0.461
Model:                            OLS   Adj. R-squared:                  0.454
Method:                 Least Squares   F-statistic:                     63.43
Date:                Mon, 31 May 2021   Prob (F-statistic):           1.07e-92
Time:                        22:01:42   Log-Likelihood:                -2935.4
No. Observations:                 753   AIC:                             5893.
Df Residuals:                     742   BIC:                             5944.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
In

107242.58714235995

## Backward stepwise selection

In [376]:
# Modelling with all variables
# columns = ['process', 'brewing_method', 'roast', 'grind', 'origin', 'percentage_of_arabica', 'pure_arabica', 'roast_brew', 'Washed', 'Natural', 'Fermented_traditional',                       'Fermented_closedtank', 'process_general', 'region_of_origin']

# for idx, col in enumerate(columns):
#     if idx == 0:
#         all_columns = col
#     else:
#         all_columns = all_columns + ' + ' + col
# print(all_columns)

model_step1 = smf.ols(formula="price_per_kg ~ C(process) + C(brewing_method) + C(roast) + C(grind) + C(origin) + percentage_of_arabica + C(pure_arabica) + C(roast_brew) + C(Washed) + C(Natural) + C(Fermented_traditional) + C(Fermented_closedtank) + C(process_general) + C(region_of_origin)", data=train_df).fit()
print(model_step1.summary())


                            OLS Regression Results                            
Dep. Variable:           price_per_kg   R-squared:                       0.595
Model:                            OLS   Adj. R-squared:                  0.541
Method:                 Least Squares   F-statistic:                     11.08
Date:                Mon, 31 May 2021   Prob (F-statistic):           7.25e-84
Time:                        22:15:41   Log-Likelihood:                -2827.8
No. Observations:                 753   AIC:                             5834.
Df Residuals:                     664   BIC:                             6245.
Df Model:                          88                                         
Covariance Type:            nonrobust                                         
                                                                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------

In [377]:
model_step1.pvalues.sort_values(ascending=False)

C(origin)[T.Salwador, Nikaragua]               9.910767e-01
C(origin)[T.Brazylia, Gwatemala, Kostaryka]    9.910767e-01
C(origin)[T.Kenia, Indie]                      9.838365e-01
C(origin)[T.Indie]                             9.511700e-01
C(Fermented_traditional)[T.True]               9.435251e-01
                                                   ...     
C(pure_arabica)[T.True]                        1.359930e-03
C(origin)[T.Kenia]                             1.189321e-03
C(process)[T.CRYO]                             1.019602e-03
C(roast_brew)[T.medium_drip, espresso]         1.443895e-04
C(grind)[T.ground]                             4.715045e-07
Length: 109, dtype: float64

In [384]:
model_step2 = smf.ols(formula="price_per_kg ~ C(process) + C(brewing_method) + C(roast) + C(grind) + percentage_of_arabica + C(pure_arabica) + C(roast_brew) + C(Washed) + C(Natural) + C(Fermented_traditional) + C(Fermented_closedtank) + C(process_general) + C(region_of_origin)", data=train_df).fit()
print(model_step2.summary()) # there was a decrease in intercept p-value by 0.052

                            OLS Regression Results                            
Dep. Variable:           price_per_kg   R-squared:                       0.516
Model:                            OLS   Adj. R-squared:                  0.489
Method:                 Least Squares   F-statistic:                     19.48
Date:                Mon, 31 May 2021   Prob (F-statistic):           2.36e-87
Time:                        22:22:44   Log-Likelihood:                -2894.9
No. Observations:                 753   AIC:                             5870.
Df Residuals:                     713   BIC:                             6055.
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
                                                                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------

In [385]:
model_step2.pvalues.sort_values(ascending=False)

C(brewing_method)[T.drip, espresso]                           0.997663
C(process)[T.Yellow Bourbon]                                  0.988286
C(process_general)[T.Experimental]                            0.916494
C(process)[T.Experimental]                                    0.916494
C(process)[T.Anaerobic]                                       0.915922
C(process_general)[T.Natural]                                 0.883215
C(process)[T.Natural]                                         0.883215
C(process_general)[T.Semi-washed]                             0.882239
C(process)[T.Semi-washed]                                     0.882239
C(process)[T.Fermentacja kontrolowana]                        0.872476
C(Natural)[T.True]                                            0.837632
C(process_general)[T.Hybrid]                                  0.759918
C(process_general)[T.Fermented]                               0.741949
C(Fermented_traditional)[T.True]                              0.739063
C(roas

In [387]:
model_step3A = smf.ols(formula="price_per_kg ~ C(process) + C(roast) + C(grind) + percentage_of_arabica + C(pure_arabica) + C(roast_brew) + C(Washed) + C(Natural) + C(Fermented_traditional) + C(Fermented_closedtank) + C(process_general) + C(region_of_origin)", data=train_df).fit()
print(model_step3A.summary())

                            OLS Regression Results                            
Dep. Variable:           price_per_kg   R-squared:                       0.516
Model:                            OLS   Adj. R-squared:                  0.489
Method:                 Least Squares   F-statistic:                     19.48
Date:                Mon, 31 May 2021   Prob (F-statistic):           2.36e-87
Time:                        22:23:12   Log-Likelihood:                -2894.9
No. Observations:                 753   AIC:                             5870.
Df Residuals:                     713   BIC:                             6055.
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
                                                                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------

In [388]:
model_step3B = smf.ols(formula="price_per_kg ~ C(brewing_method) + C(roast) + C(grind) + percentage_of_arabica + C(pure_arabica) + C(roast_brew) + C(Washed) + C(Natural) + C(Fermented_traditional) + C(Fermented_closedtank) + C(process_general) + C(region_of_origin)", data=train_df).fit()
print(model_step3B.summary()) # a significant further decrease in the pvalue

                            OLS Regression Results                            
Dep. Variable:           price_per_kg   R-squared:                       0.495
Model:                            OLS   Adj. R-squared:                  0.477
Method:                 Least Squares   F-statistic:                     26.36
Date:                Mon, 31 May 2021   Prob (F-statistic):           2.42e-89
Time:                        22:23:50   Log-Likelihood:                -2910.5
No. Observations:                 753   AIC:                             5877.
Df Residuals:                     725   BIC:                             6007.
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                                                                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------

In [389]:
model_step3B.pvalues.sort_values(ascending=False)

C(brewing_method)[T.drip, espresso]                           9.907722e-01
C(roast_brew)[T.light_drip, espresso]                         7.615804e-01
C(brewing_method)[T.espresso]                                 7.221268e-01
C(roast)[T.medium]                                            7.120607e-01
C(roast_brew)[T.light_drip (alternative brewing methods)]     6.544166e-01
C(Natural)[T.True]                                            6.535988e-01
C(roast_brew)[T.medium_drip (alternative brewing methods)]    5.414785e-01
C(pure_arabica)[T.True]                                       4.831147e-01
C(region_of_origin)[T.Europe]                                 4.284620e-01
C(roast_brew)[T.dark_drip, espresso]                          3.679040e-01
C(process_general)[T.Hybrid]                                  1.849210e-01
C(region_of_origin)[T.Latam]                                  1.775828e-01
C(region_of_origin)[T.Asia]                                   1.767634e-01
C(roast_brew)[T.dark_espr

In [393]:
model_step4 = smf.ols(formula="price_per_kg ~ C(brewing_method) + C(roast) + C(grind) + percentage_of_arabica + C(pure_arabica) + C(Washed) + C(Natural) + C(Fermented_traditional) + C(Fermented_closedtank) + C(process_general) + C(region_of_origin)", data=train_df).fit()
print(model_step4.summary()) # a significant further decrease in the pvalue

                            OLS Regression Results                            
Dep. Variable:           price_per_kg   R-squared:                       0.490
Model:                            OLS   Adj. R-squared:                  0.474
Method:                 Least Squares   F-statistic:                     30.51
Date:                Mon, 31 May 2021   Prob (F-statistic):           8.32e-91
Time:                        22:25:22   Log-Likelihood:                -2914.2
No. Observations:                 753   AIC:                             5876.
Df Residuals:                     729   BIC:                             5987.
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
In

In [395]:
model_step5 = smf.ols(formula="price_per_kg ~ C(brewing_method) + C(roast) + C(grind) + C(pure_arabica) + C(Natural) + C(Fermented_traditional) + C(Fermented_closedtank) + C(process_general) + C(region_of_origin)", data=train_df).fit()
print(model_step5.summary()) # a significant further decrease in the pvalue

                            OLS Regression Results                            
Dep. Variable:           price_per_kg   R-squared:                       0.481
Model:                            OLS   Adj. R-squared:                  0.466
Method:                 Least Squares   F-statistic:                     32.29
Date:                Mon, 31 May 2021   Prob (F-statistic):           1.46e-89
Time:                        22:26:42   Log-Likelihood:                -2920.9
No. Observations:                 753   AIC:                             5886.
Df Residuals:                     731   BIC:                             5988.
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
In

In [399]:
model_step5 = smf.ols(formula="price_per_kg ~ C(brewing_method) + C(roast) + C(grind) + C(pure_arabica) + C(Fermented_traditional) + C(Fermented_closedtank)", data=train_df).fit()
print(model_step5.summary()) # a significant further decrease in the pvalue

                            OLS Regression Results                            
Dep. Variable:           price_per_kg   R-squared:                       0.447
Model:                            OLS   Adj. R-squared:                  0.441
Method:                 Least Squares   F-statistic:                     75.18
Date:                Mon, 31 May 2021   Prob (F-statistic):           1.51e-90
Time:                        22:27:46   Log-Likelihood:                -2945.0
No. Observations:                 753   AIC:                             5908.
Df Residuals:                     744   BIC:                             5950.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
In

In [406]:
model_step5 = smf.ols(formula="price_per_kg ~ C(brewing_method) + C(roast) + C(grind) + C(pure_arabica) + C(Fermented_closedtank)", data=train_df).fit()
print(model_step5.summary())
model_step5.ssr

                            OLS Regression Results                            
Dep. Variable:           price_per_kg   R-squared:                       0.446
Model:                            OLS   Adj. R-squared:                  0.441
Method:                 Least Squares   F-statistic:                     85.75
Date:                Mon, 31 May 2021   Prob (F-statistic):           2.73e-91
Time:                        22:30:07   Log-Likelihood:                -2945.5
No. Observations:                 753   AIC:                             5907.
Df Residuals:                     745   BIC:                             5944.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
In

110157.32740949612

In [407]:
train_df_important = train_df[['brewing_method', 'roast', 'pure_arabica', 'grind', 'Fermented_closedtank']]
train_df_important

,brewing_method,roast,pure_arabica,grind,Fermented_closedtank
837,espresso,medium,True,beans,False
697,espresso,medium,True,beans,False
96,drip (alternative brewing methods),light,True,beans,False
677,espresso,medium,True,beans,False
823,espresso,dark,True,beans,True
...,...,...,...,...,...
739,espresso,dark,False,ground,True
791,espresso,dark,False,beans,False
78,drip (alternative brewing methods),light,True,beans,False
250,drip (alternative brewing methods),light,True,beans,False


In [408]:
train_df_important.corr()

,pure_arabica,Fermented_closedtank
pure_arabica,1.000000,0.003429
Fermented_closedtank,0.003429,1.000000


In [ ]:
# import plotly.graph_objects as go

# coffee_df['arabica_fitted'] = model.fittedvalues

# fig = go.Figure()

# fig.add_trace(go.Scatter(
#     x=coffee_df["percentage_of_arabica"], y=coffee_df["price_per_kg"], name="percentage_of_arabica vs price_per_kg (million sq km)", mode="markers"))
# fig.add_trace(go.Scatter(
#     x=coffee_df["percentage_of_arabica"], y=coffee_df["arabica_fitted"], name="Fitted Regression Line"))
# fig.update_layout(title="Regression line of percentage_of_arabica vs price_per_kg (million sq km)", xaxis_title="percentage_of_arabica",
#     yaxis_title="price_per_kg", height=800, width=950)
# fig.show()

In [409]:
import plotly.express as px

px.imshow(train_df_important.corr(), color_continuous_scale='Agsunset', title="Correlation between coffee variables")

### Observations:
* variables chosen in the forward and backward stepwise selection differ by 1 variable:
    - Forward: brewing_method, roast, pure_arabica, grind
    - Backward: brewing_method, roast, pure_arabica, grind, Fermented_closedtank
* the ssr for each model is different with backward being smaller:
    - Forward:  111039.32822297813
    - Backward: 110157.32740949612

In [ ]:
# TODO: Interactions and transformations